In [1]:
from utils.compute_overlap import compute_overlap
from utils.visualization import draw_detections, draw_annotations

import numpy as np
import cv2
import progressbar
import time

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
assert (callable(progressbar.progressbar)), "Using wrong progressbar module, install 'progressbar2' instead."


def _compute_ap(recall, precision):
    """
    Compute the average precision, given the recall and precision curves.

    Code originally from https://github.com/rbgirshick/py-faster-rcnn.

    Args:
        recall: The recall curve (list).
        precision: The precision curve (list).

    Returns:
        The average precision as computed in py-faster-rcnn.

    """
    # correct AP calculation
    # first append sentinel values at the end
    mrec = np.concatenate(([0.], recall, [1.]))
    mpre = np.concatenate(([0.], precision, [0.]))

    # compute the precision envelope
    for i in range(mpre.size - 1, 0, -1):
        mpre[i - 1] = np.maximum(mpre[i - 1], mpre[i])

    # to calculate area under PR curve, look for points
    # where X axis (recall) changes value
    i = np.where(mrec[1:] != mrec[:-1])[0]

    # and sum (delta recall) * prec
    ap = np.sum((mrec[i + 1] - mrec[i]) * mpre[i + 1])
    return ap


def _get_detections(generator, model, score_threshold=0.05, max_detections=1, visualize=False):
    """
    Get the detections from the model using the generator.

    The result is a list of lists such that the size is:
        all_detections[num_images][num_classes] = detections[num_class_detections, 5]

    Args:
        generator: The generator used to run images through the model.
        model: The model to run on the images.
        score_threshold: The score confidence threshold to use.
        max_detections: The maximum number of detections to use per image.
        save_path: The path to save the images with visualized detections to.

    Returns:
        A list of lists containing the detections for each image in the generator.

    """
    all_detections = [[None for i in range(generator.num_classes()) if generator.has_label(i)] for j in
                      range(generator.size())]

    for i in progressbar.progressbar(range(generator.size()), prefix='Running network: '):
        image = generator.load_image(i)
        src_image = image.copy()
        h, w = image.shape[:2]

        anchors = generator.anchors
        image, scale = generator.preprocess_image(image)

        # run network
        boxes, scores, *_, labels = model.predict_on_batch([np.expand_dims(image, axis=0)])
        boxes /= scale
        boxes[:, :, 0] = np.clip(boxes[:, :, 0], 0, w - 1)
        boxes[:, :, 1] = np.clip(boxes[:, :, 1], 0, h - 1)
        boxes[:, :, 2] = np.clip(boxes[:, :, 2], 0, w - 1)
        boxes[:, :, 3] = np.clip(boxes[:, :, 3], 0, h - 1)

        # select indices which have a score above the threshold
        indices = np.where(scores[0, :] > score_threshold)[0]

        # select those scores
        scores = scores[0][indices]

        # find the order with which to sort the scores
        scores_sort = np.argsort(-scores)[:max_detections]
#         print(len(scores_sort))
#         print(scores_sort)

        # select detections
        # (n, 4)
        image_boxes = boxes[0, indices[scores_sort], :]
        # (n, )
        image_scores = scores[scores_sort]
        # (n, )
        image_labels = labels[0, indices[scores_sort]]
        # (n, 6)
        detections = np.concatenate(
            [image_boxes, np.expand_dims(image_scores, axis=1), np.expand_dims(image_labels, axis=1)], axis=1)
#         print(detections)

#         draw_annotations(src_image, generator.load_annotations(i), label_to_name=generator.label_to_name)
#         draw_detections(src_image, detections[:5, :4], detections[:5, 4], detections[:5, 5].astype(np.int32),
#                             label_to_name=generator.label_to_name,
#                             score_threshold=score_threshold)

#         # cv2.imwrite(os.path.join(save_path, '{}.png'.format(i)), raw_image)
#         cv2.namedWindow('{}'.format(i), cv2.WINDOW_NORMAL)
#         cv2.imshow('{}'.format(i), src_image)
#         cv2.waitKey(0)

        # copy detections to all_detections
        for class_id in range(generator.num_classes()):
            all_detections[i][class_id] = detections[detections[:, -1] == class_id, :-1]
#         print(all_detections)
        
        f=open("/ssd3/u1/NBI_NET/EfficientDet-master/bbox.txt",'a+')
#         print("image_boxes= ",image_boxes)
#         print(image_boxes.shape)
        count=0
        bbox_temp=[]
        temp_single_position=0
#         print("max_detections= ",max_detections)
        temp_xmin,temp_ymin,temp_xmax,temp_ymax=[],[],[],[]
        
#         for i in range(len(scores_sort)):
#             temp_xmin.append(image_boxes[i][0])
#             temp_ymin.append(image_boxes[i][1])
#             temp_xmax.append(image_boxes[i][2])
#             temp_ymax.append(image_boxes[i][3])
#         bbox_temp.append(min(temp_xmin))
#         bbox_temp.append(min(temp_ymin))
#         if max(temp_xmax)>480:
#             bbox_temp.append(480)
#         else:
#             bbox_temp.append(max(temp_xmax))
#         if max(temp_ymax)>480:
#             bbox_temp.append(480)
#         else:
#             bbox_temp.append(max(temp_ymax))
        
        
        for i in range(4):
            for j in range(len(scores_sort)):                
                temp_single_position=temp_single_position+image_boxes[j][count]
            temp_single_position=temp_single_position/(max_detections)
            if temp_single_position>480:
                temp_single_position=480
            bbox_temp.append(temp_single_position)
            count+=1
        bbox_temp=np.array(bbox_temp)
#         print(bbox_temp.shape)
#         print("image_boxes_append",bbox_temp)
        count=0
        for i in range(4):
            if count!=3:
                f.write(str(bbox_temp[count])+',')
            else:
                f.write(str(bbox_temp[count]))
            count+=1
        f.write('\n')
        f.close()

    return all_detections


def _get_annotations(generator):
    """
    Get the ground truth annotations from the generator.

    The result is a list of lists such that the size is:
        all_annotations[num_images][num_classes] = annotations[num_class_annotations, 5]

    Args:
        generator: The generator used to retrieve ground truth annotations.

    Returns:
        A list of lists containing the annotations for each image in the generator.

    """
    all_annotations = [[None for i in range(generator.num_classes())] for j in range(generator.size())]

    for i in progressbar.progressbar(range(generator.size()), prefix='Parsing annotations: '):
        # load the annotations
        annotations = generator.load_annotations(i)

        # copy detections to all_annotations
        for label in range(generator.num_classes()):
            if not generator.has_label(label):
                continue

            all_annotations[i][label] = annotations['bboxes'][annotations['labels'] == label, :].copy()

    return all_annotations


def evaluate(
        generator,
        model,
        iou_threshold=0.5,
        score_threshold=0.01,
        max_detections=5,
        visualize=False,
        epoch=0
):
    """
    Evaluate a given dataset using a given model.

    Args:
        generator: The generator that represents the dataset to evaluate.
        model: The model to evaluate.
        iou_threshold: The threshold used to consider when a detection is positive or negative.
        score_threshold: The score confidence threshold to use for detections.
        max_detections: The maximum number of detections to use per image.
        visualize: Show the visualized detections or not.

    Returns:
        A dict mapping class names to mAP scores.

    """
    # gather all detections and annotations
    all_detections= _get_detections(generator, model, score_threshold=score_threshold, max_detections=max_detections,
                                     visualize=visualize)
    all_annotations = _get_annotations(generator)
    average_precisions = {}
    num_tp = 0
    num_fp = 0

    
    # process detections and annotations
    for label in range(generator.num_classes()):
        if not generator.has_label(label):
            continue

        false_positives = np.zeros((0,))
        true_positives = np.zeros((0,))
        scores = np.zeros((0,))
        num_annotations = 0.0

        for i in range(generator.size()):
            detections = all_detections[i][label]
            annotations = all_annotations[i][label]
            num_annotations += annotations.shape[0]
            detected_annotations = []

            for d in detections:
                scores = np.append(scores, d[4])

                if annotations.shape[0] == 0:
                    false_positives = np.append(false_positives, 1)
                    true_positives = np.append(true_positives, 0)
                    continue
                overlaps = compute_overlap(np.expand_dims(d, axis=0), annotations)
                assigned_annotation = np.argmax(overlaps, axis=1)
                max_overlap = overlaps[0, assigned_annotation]

                if max_overlap >= iou_threshold and assigned_annotation not in detected_annotations:
                    false_positives = np.append(false_positives, 0)
                    true_positives = np.append(true_positives, 1)
                    detected_annotations.append(assigned_annotation)
                else:
                    false_positives = np.append(false_positives, 1)
                    true_positives = np.append(true_positives, 0)

        # no annotations -> AP for this class is 0 (is this correct?)
        if num_annotations == 0:
            average_precisions[label] = 0, 0
            continue

        # sort by score
        indices = np.argsort(-scores)
        false_positives = false_positives[indices]
        true_positives = true_positives[indices]

        # compute false positives and true positives
        false_positives = np.cumsum(false_positives)
        true_positives = np.cumsum(true_positives)

        if false_positives.shape[0] == 0:
            num_fp += 0
        else:
            num_fp += false_positives[-1]
        if true_positives.shape[0] == 0:
            num_tp += 0
        else:
            num_tp += true_positives[-1]

        # compute recall and precision
        recall = true_positives / num_annotations
        precision = true_positives / np.maximum(true_positives + false_positives, np.finfo(np.float64).eps)

        # compute average precision
        average_precision = _compute_ap(recall, precision)
        average_precisions[label] = average_precision, num_annotations
    print('num_fp={}, num_tp={}'.format(num_fp, num_tp))

    return average_precisions,recall,precision,iou_threshold,max_detections


if __name__ == '__main__':
    from generators.pascal import PascalVocGenerator
    from model import efficientdet
    import os
    f=open("/ssd3/u1/NBI_NET/EfficientDet-master/bbox.txt","w")
    f.close()

    max_detections=100
    phi = 1
    weighted_bifpn = False
    common_args = {
        'batch_size': 1,
        'phi': phi,
    }
    test_generator = PascalVocGenerator(
        'datasets/VOCdevkit/VOC2007',
        'test',
        shuffle_groups=False,
        skip_truncated=False,
        skip_difficult=True,
        **common_args
    )
    model_path = 'checkpoints/fpi1-test01_R_ann4/pascal_04_0.4206_1.0106_879.h5'#fpi1_ann4
#     model_path = 'checkpoints/fpi1-test02_R_ann2/pascal_02_0.0912_0.0791.h5'#fpi1_ann2
#     model_path = 'checkpoints/fpi3-test11_R_ann4/best_pascal_06_0.3386_0.7871_642.h5'#fpi3_ann4
#     model_path = 'checkpoints/fpi3-test12_R_ann2/pascal_03_0.0493_0.9485.h5'#fpi3_ann2
    input_shape = (test_generator.image_size, test_generator.image_size)
    anchors = test_generator.anchors
    num_classes = test_generator.num_classes()
    
    model, prediction_model = efficientdet(phi=phi, num_classes=num_classes, weighted_bifpn=weighted_bifpn)
    
    prediction_model.load_weights(model_path, by_name=True)
    tStart = time.time()
    average_precisions,recall,precison,iou_threshold,max_detections= evaluate(test_generator, prediction_model,visualize=False)
    tEnd = time.time()
    print("It cost %f sec" % (tEnd - tStart))
    # compute per class average precision
    total_instances = []
    precisions = []
    for label, (average_precision, num_annotations) in average_precisions.items():
        print('{:.0f} instances of class'.format(num_annotations), test_generator.label_to_name(label),
              'with average precision: {:.4f}'.format(average_precision))
        total_instances.append(num_annotations)
        precisions.append(average_precision)
    mean_ap = sum(precisions) / sum(x > 0 for x in total_instances)
    print("Max detections: ",max_detections)
    print("IOU Threshold: ",iou_threshold)
    print('mAP: {:.4f}'.format(mean_ap))
    print('recall: ',np.mean(recall))
    print('precison: ',np.mean(precison))

Instructions for updating:
`normal` is a deprecated alias for `truncated_normal`
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


100% (13 of 13) |########################| Elapsed Time: 0:00:05 Time:  0:00:05
100% (13 of 13) |########################| Elapsed Time: 0:00:00 Time:  0:00:00


num_fp=52.0, num_tp=13.0
It cost 5.783306 sec
13 instances of class cancer with average precision: 0.8797
Max detections:  5
IOU Threshold:  0.5
mAP: 0.8797
recall:  0.8757396449704141
precison:  0.48030083092849124
